<a href="https://colab.research.google.com/github/huyen1607/California-Housing-Price-Prediction/blob/main/California_Housing_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
d = []
while(1):
    d.append(1)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#수치계산에에 사용하는는 라이브러리
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

#선형모델델 추정정 라이브러리
import statsmodels.formula.api as smf
import statsmodels.api as sm

#다층층 퍼셉트론용용 애플리케이션
from sklearn.neural_network import MLPClassifier

#샘플데이터터 불러오기
from sklearn.datasets import fetch_california_housing

#테스트데이터와와 훈련련 데이터터 분리
from sklearn.model_selection import train_test_split

#데이터터 표준화용용 라이브러리
from sklearn.preprocessing import StandardScaler

%precision 3

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


'%.3f'

In [ ]:
#데이터터 불러오기
housing = fetch_california_housing()

In [ ]:
#독립변수
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [ ]:
#종속변수
housing.target_names

['MedHouseVal']

In [ ]:
housing.target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [ ]:
price_df = pd.DataFrame(housing.target, columns=['MedHouseVal'])
print(price_df)

       MedHouseVal
0            4.526
1            3.585
2            3.521
3            3.413
4            3.422
...            ...
20635        0.781
20636        0.771
20637        0.923
20638        0.847
20639        0.894

[20640 rows x 1 columns]


In [ ]:
price_df.describe()

,MedHouseVal
count,20640.000000
mean,2.068558
std,1.153956
min,0.149990
25%,1.196000
50%,1.797000
75%,2.647250
max,5.000010


In [ ]:
#MedHouseval는 중앙값보다 크면 High_Price이고 중앙값보다 적으면 Low_Price이다
#MedHouseval 1.797보다 크면 High Price이고 1로 변화,넘머지는 Low_Price이고 0로 변화
target_new = (housing.target > 1.797).astype(int)
target_new

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
#독립변수 Latitude과 Longtitude만 가져오기기
X=housing.data[0:20640,6:]

y=target_new

print('독립변수 행 수,열 수: ', X.shape)
print('종속변수 행 수,열 수: ', y.shape)

독립변수 행 수,열 수:  (20640, 2)
종속변수 행 수,열 수:  (20640,)


In [ ]:
#데이터를를 훈련련 데이터와와 테스트 데이터로로 분리
X_train,X_test, y_train, y_test = train_test_split(X,y,random_state=1234)

print('독립변수 행 수,열 수: ', X_train.shape)
print('종속변수 행 수,열 수: ', y_train.shape)

독립변수 행 수,열 수:  (15480, 2)
종속변수 행 수,열 수:  (15480,)


In [ ]:
#종속변수
y_train[0:10]

array([0, 0, 1, 1, 0, 0, 0, 1, 0, 1])

In [ ]:
#데이터터 정리
#독립변수수 데이터프레임
X_train_df = pd.DataFrame(X_train, columns=['Latitude','Longtitude'])
#종속변수수 데이터프레임
y_train_df = pd.DataFrame({"Price":y_train})
#데이터프레임임 결합
housing_train_df = pd.concat([y_train_df,X_train_df],axis=1)
#결과과 출력
print(housing_train_df.head(5))

   Price  Latitude  Longtitude
0      0     34.57     -117.93
1      0     38.14     -121.28
2      1     33.89     -117.86
3      1     37.70     -122.14
4      0     36.37     -119.65


In [ ]:
#모델로 만들기
logi_mod_full = smf.glm('Price ~ Latitude + Longtitude',data=housing_train_df,
                        family=sm.families.Binomial()).fit()
logi_mod_Latitude = smf.glm('Price ~ Latitude',data=housing_train_df,
                        family=sm.families.Binomial()).fit()
logi_mod_Longtitude = smf.glm('Price ~ Longtitude',data=housing_train_df,
                        family=sm.families.Binomial()).fit()
logi_mod_null = smf.glm('Price ~ 1',data=housing_train_df,
                        family=sm.families.Binomial()).fit()
#AIC비교
print('full', logi_mod_full.aic.round(3))
print('Latitude', logi_mod_Latitude.aic.round(3))
print('Longtitude', logi_mod_Longtitude.aic.round(3))
print('null', logi_mod_null.aic.round(3))

full 17183.208
Latitude 21152.514
Longtitude 21438.724
null 21461.648


In [ ]:
#모델추정정 결과
logi_mod_full.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-154.2069,3.155,-48.878,0.000,-160.390,-148.023
Latitude,-1.8287,0.037,-49.743,0.000,-1.901,-1.757
Longtitude,-1.8333,0.037,-49.839,0.000,-1.905,-1.761


In [ ]:
#모델 정확도도 테스트
#데이터 정리
X_test_df = pd.DataFrame(X_test, columns=["Latitude","Longtitude"])

#피팅팅 예측
logi_fit = logi_mod_full.fittedvalues.round(0)
logi_pred = logi_mod_full.predict(X_test_df).round(0)

#정답 수
true_train=np.sum(logi_fit == (y_train))
true_test=np.sum(logi_pred == (y_test))

#적중률
result_train = true_train/len(y_train)
result_test = true_test / len(y_test)

#결과출력
print("훈련데이터 적줄률:",result_train)
print("테스트 데이터 적줄률:", result_test)

훈련데이터 적줄률: 0.7172480620155038
테스트 데이터 적줄률: 0.7304263565891473


In [ ]:
#신경망 구현

In [ ]:
#표준환 준비
scaler = StandardScaler()
scaler.fit(X_train)
#표준화
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#훈련데이터의 독립변수의 표준편차 1 확인
np.std(X_train_scaled, axis=0)

array([1., 1.])

In [ ]:
#테이스터의 독립변수의 표준편차 1 확인
np.std(X_test_scaled, axis=0)

array([0.995, 1.005])

In [ ]:
nnet = MLPClassifier(
    hidden_layer_sizes = (100,100),
    alpha = 0.07,
    max_iter = 10000,
    random_state = 0
)
nnet.fit(X_train_scaled, y_train)

#정확도
print("훈련데이터 적줄률:", nnet.score(X_train_scaled, y_train))
print("테스트 데이터 적줄률:", nnet.score(X_test_scaled, y_test))

훈련데이터 적줄률: 0.7850129198966408
테스트 데이터 적줄률: 0.790891472868217


In [ ]:
#로지스틱 회기보다 신경망은  정확도는 더 높다.